In [58]:
import pandas as pd
import numpy as np

# 1. Column mapping

In [59]:
INPUT_FILE = 'data/FEMA NHS/2023/fema_nhs_data_2023.xlsx'
OUTPUT_DIR = 'data/processed'
FLORIDA_FIPS = 12

In [ ]:
HURRICANE_COLUMNS = {
    # Identifiers
    'respid': 0,
    'sample': 1,
    'weight': 2,
    
    # Geography
    'state': 3,
    'division': 4,
    'region': 5,
    'zip': 6,
    'county': 7,
    
    # Hurricane Experience & Risk
    'hrcn_aware': 8,
    'hrcn_risk_perception': 9,
    'hrcn_experience': 10,
    'hrcn_prepactions': 11,
    'hrcn_safe_shelter': 12,
    'hrcn_impacts': 29,
    'hrcn_shutters': 35,
    'hrcn_prep_specific': 36,
    
    # Preparedness & Psychology
    'hrcn_prep_efficacy': 26,
    'hrcn_prep_confidence': 27,
    'hrcn_preparedness_stage': 28,
    
    # Calculated preparedness fields
    'hrcn_action_1plus': 19,
    'hrcn_action_3plus': 20,
    'hrcn_influencer_aware': 21,
    'hrcn_influencer_exp': 22,
    'hrcn_influencer_efficacy': 23,
    'hrcn_influencer_risk': 24,
    'hrcn_prep_3cat': 57,
    'hrcn_prep_2cat': 58,
    
    # Demographics
    'age': 59,
    'sex': 60,
    'education': 62,
    'ethnicity': 64,
    'race': 65,
    'disability': 66,
    'caretaker': 67,
    'numadult': 68,
    'numchild': 69,
    'numchild_school': 70,
    'school_plan_aware': 71,
    'primary_language': 72,
    'income': 79,
    'employment': 80,
    
    # Location & Housing
    'homeownership': 74,
    'mortgage': 76,
    'hometype': 77,
    
    # Calculated fields
    'ses_score': 106,
    'english_primary': 107,
    'rurality': 108,
    'mortgage_4cat': 109,
    'income_3cat': 110,
}

In [61]:
CORE_SURVEY_COLUMNS = {
    'respid': 0,  # Should match Hurricane sheet for merging
    'homeowners_insurance': 264,
    'hazard_insurance': 265,
}

# 2. Loading data

In [62]:
hurricane = pd.read_excel('data/FEMA NHS/2023/fema_nhs_data_2023.xlsx', 
                          sheet_name = 'Hurricane',
                          header = 0)

core = pd.read_excel('data/FEMA NHS/2023/fema_nhs_data_2023.xlsx', 
                          sheet_name = 'Core Survey',
                          header = 0)

In [63]:
# Function selecting column based on maps

def select_columns(df, column_map):
    cols = {}
    for x, col_id in column_map.items():
        cols[x] = df.iloc[:, col_id]
    return pd.DataFrame(cols)

# 3. Cleaning functinons

In [ ]:
# Function for cleaning both core survey and hurricane

def clean(df):

    df_clean = df.copy()

    num_cols = ['respid', 'weight', 'numadult', 'numchild', 'zip']
    for col in num_cols:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    if 'hrcn_experience' in df_clean.columns:
        df_clean['hrcn_exp_flag'] = (df_clean['hrcn_experience'] == 1).astype(int)
    
    if 'numchild' in df_clean.columns:
        df_clean['has_children'] = (df_clean['numchild'] > 0).astype(int)
    
    if 'numchild_school' in df_clean.columns:
        df_clean['children_in_school'] = (df_clean['numchild_school'] == 1).astype(int)
    
    return df_clean
        
    

In [65]:
# Function for merging core with hurricane

def merge(df1, df2):
    insurance_cols = select_columns(df2, CORE_SURVEY_COLUMNS) 

    merged_df = df1.merge(
        insurance_cols,
        on = 'respid',
        how = 'left',
        suffixes = ('', '_core')
    )
    return merged_df

# 4. Geographic filtering

In [ ]:
def filter(df):
    fl_mask = df['state'].str.contains('Florida', case=False, na=False)
    df_fl = df[fl_mask].copy()

    return df_fl

# 5. Creating new composite scores (double check)

In [67]:
def create_composite(df):
    df_new = df.copy()

    # Preparedness score
    prep_cols = ['hrcn_iprepefficacy', 'hrcn_confidence', 
                       'hrcn_atleast1_prepaction', 'hrcn_iawareness']
    available_prep = [col for col in prep_cols if col in df_new.columns]
    if available_prep:
        df_new['preparedness_score'] = df_new[available_prep].mean(axis=1)

    # Risk score
    risk_cols = ['hrcn_iperception', 'hrcn_awareness', 'hrcn_exp']
    available_risk = [col for col in risk_cols if col in df_new.columns]

    if available_risk:
        df_new['risk_awareness_score'] = df_derived[available_risk].mean(axis=1)

    return df_new

# 6. Putting everything together + exporting

In [68]:
hurricane_selected = select_columns(hurricane, HURRICANE_COLUMNS)
print(f"Hurricane columns selected: {len(hurricane_selected.columns)}")

hurricane_clean = clean(hurricane_selected)
merged_data = merge(hurricane_clean, core)

fl_data = filter(merged_data)

fl_data = create_composite(fl_data)

Hurricane columns selected: 49
There are 225 records.


In [69]:
fl_data.head()

,respid,sample,weight,state,division,region,zip,county,hrcn_aware,hrcn_risk_perception,...,ses_score,english_primary,rurality,mortgage_4cat,income_3cat,hrcn_exp_flag,has_children,children_in_school,homeowners_insurance,hazard_insurance
2,1011.0,Hurricane,0.665038,Florida,South Atlantic,South,32746.0,Seminole,Yes,Very likely,...,Not disadvantaged,Primarily English-Speaking,Urban,"$1,001 to $2,000",Less than $50k,0,0,0,Yes,Blank
3,1012.0,Hurricane,1.072655,Florida,South Atlantic,South,33324.0,Broward,Yes,Likely,...,Not disadvantaged,Primarily English-Speaking,Urban,"$1,001 to $2,000","$50,000 to $99,999",0,0,0,Yes,Blank
8,1017.0,Hurricane,0.213886,Florida,South Atlantic,South,32132.0,Volusia,No,Very likely,...,Disadvantaged,Primarily English-Speaking,Urban,"$1,000 or Less",Less than $50k,0,0,0,No,Blank
10,1019.0,Hurricane,0.784652,Florida,South Atlantic,South,32962.0,Indian River,Yes,Very likely,...,Not disadvantaged,Primarily English-Speaking,Urban,"$1,000 or Less",Less than $50k,0,0,0,No,Blank
12,1021.0,Hurricane,0.294472,Florida,South Atlantic,South,33908.0,Lee,Yes,Likely,...,Not disadvantaged,Primarily English-Speaking,Urban,"$1,000 or Less",Less than $50k,0,0,0,Yes,Blank


In [70]:
fl_data.to_csv('data/processed/FL_NHS.csv')